## Facilities and Educational Data for Boston Public Schools
Data is acquired from here: https://data.boston.gov/dataset/buildbps-facilities-and-educational-data-for-boston-public-schools
where the terms are explained in a  pdf 

In this exercise you can practice a bit of datacleaning as well.

In [1]:
from bokeh.models import ColumnDataSource, Plot, LinearAxis, Grid, Circle, HoverTool, BoxSelectTool,\
    SaveTool, CustomJS, DatetimeAxis, LinearAxis, NumeralTickFormatter
from bokeh.models.widgets.tables import (
    DataTable, TableColumn, IntEditor
)
from bokeh.models.widgets import DataTable, TableColumn, StringFormatter,\
        NumberFormatter, StringEditor, IntEditor, NumberEditor, SelectEditor,\
        CheckboxButtonGroup, CheckboxGroup, MultiSelect, RadioButtonGroup,\
        Select, Slider, Panel, Tabs, TextInput, Paragraph, Div, Button, Dropdown

from bokeh.models.glyphs import Circle
from bokeh.io import curdoc, output_notebook, show as showio, output_file
from bokeh.plotting import show, figure
from bokeh.document import Document
from bokeh.models.layouts import Column, Row
from bokeh.embed import file_html
from bokeh.resources import INLINE
from bokeh.util.browser import view

from IPython.core.display import display, HTML
import copy
import pandas as pd
import numpy as np

In [2]:
output_notebook()

Loading BokehJS ...

Let's read the *data/buildbps.csv* file into a pandas DataFrame

In [3]:
df = pd.read_csv("data/buildbps.csv", sep=",")

### Scatter plot
Create a plot showing the *BPS_Water_Bill* (cost of water usage) as a function of the *BPS_Gas_Bill* (cost of gas usage) columns! It will show some kind of correlation

You should get a similar plot to this:

In [4]:
display(HTML(open('data/scatter-water-gas.html').read()))

In [5]:
# convert dataframe to bokeh type, create figure and add a scatter plot to it
data_source = ColumnDataSource(df)
p = figure(plot_width=400, plot_height=400)
p.scatter("BPS_Water_Bill", "BPS_Gas_Bill", source=data_source)
show(p)

2020-09-30 22:27:50,420 [241] ERROR    bokeh.core.validation.check:121: [JupyterRequire] E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "BPS_Water_Bill" (closest match: " BPS_Water_Bill "), key "y" value "BPS_Gas_Bill" (closest match: " BPS_Gas_Bill ") [renderer: GlyphRenderer(id=1037, glyph=Scatter(id='1035', ...), ...)]


### Why do we get this error?

In [6]:
# There is a space before and after. Why would anyone do something like this?
df[' BPS_Water_Bill ']

0             $-   
1        $1,685.00 
2       $20,763.00 
3        $5,227.00 
4        $5,262.00 
           ...     
136     $36,183.00 
137     $37,653.00 
138      $5,394.00 
139      $7,457.00 
140     $11,457.00 
Name:  BPS_Water_Bill , Length: 141, dtype: object

In [7]:
# Let's rename the columns
df['BPS_Water_Bill'] = df[' BPS_Water_Bill ']
df['BPS_Gas_Bill'] = df[' BPS_Gas_Bill ']

### We will need numbers, not currency format
we should convert it to numbers

In [8]:
# using a regular expression for replacing characters in the vallues
# https://en.wikipedia.org/wiki/Regular_expression
# practice at https://www.w3schools.com/python/python_regex.asp

df['BPS_Water_Bill'] = df['BPS_Water_Bill'].replace( '\$-','0', regex=True ).replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)
df['BPS_Gas_Bill'] = df['BPS_Gas_Bill'].replace( '\$-','0', regex=True ).replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)

In [9]:
# and plot it finally
data_source = ColumnDataSource(df)
p = figure(plot_width=400, plot_height=400)
p.scatter("BPS_Water_Bill", "BPS_Gas_Bill", source=data_source)
output_file("scatter-water-gas.html")
show(p)

## Circles
Let's add a 3rd dimension to the plot

In [21]:
# we can associate the size of the circles with a column of the datatable, but we need to normalize it in someway
# which column would you choose?

In [24]:
# We just randomly choose the electricity bill
df['BPS_Electric_Bill'] = df[' BPS_Electric_Bill '].replace( '\$-','0', regex=True ).replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)
data_source = ColumnDataSource(df)

In [31]:
size = df.BPS_Electric_Bill/df.BPS_Electric_Bill.max()*20 

# We either add the array of sizes to the data_source object or should avoid its usage for now
p = figure(plot_width=400, plot_height=400)
p.circle(x=df["BPS_Water_Bill"], y=df["BPS_Gas_Bill"], size=size.to_numpy(), color="navy", alpha=0.5) #, source=data_source)
show(p)

In [34]:
# Here we add the size to the data_source vaiable
df['eb_size'] = df.BPS_Electric_Bill/df.BPS_Electric_Bill.max()*20 
data_source = ColumnDataSource(df)

p = figure(plot_width=400, plot_height=400)
p.circle(x='BPS_Water_Bill', y='BPS_Gas_Bill', size='eb_size', color="navy", alpha=0.5 , source=data_source)
show(p)

### Add a title to it

In [27]:
p = figure(plot_width=400, plot_height=400, title="Expenses of the buildings")

p.circle(x=df["BPS_Water_Bill"], y=df["BPS_Gas_Bill"], size=size.to_numpy(), color="navy", alpha=0.5) #, source=data_source)
p.xaxis.formatter = NumeralTickFormatter()
p.xaxis.axis_label = "Water Bill"
p.yaxis.formatter = NumeralTickFormatter(format='$0,0')
p.yaxis.axis_label = "Gas Bill"
show(p)

### Colors
Let's add a 4th dimension to the plot with colors

In [35]:
size = df.BPS_Electric_Bill/df.BPS_Electric_Bill.max()*20
colors = df.SMMA_FA_Existing_PV.apply(lambda x: 'red' if x=="No" else 'green')

# We either add the array of sizes to the data_source object or should avoid its usage for now
p = figure(plot_width=400, plot_height=400, title="Expenses of the buildings")
p.circle(x=df["BPS_Water_Bill"], y=df["BPS_Gas_Bill"], size=size.to_numpy(), color=colors, alpha=0.5)
p.xaxis.formatter = NumeralTickFormatter(format='$0,0'); p.xaxis.axis_label = "Water Bill"
p.yaxis.formatter = NumeralTickFormatter(format='$0,0'); p.yaxis.axis_label = "Gas Bill"
show(p)

### And add legends too

In [37]:
data_source = ColumnDataSource(dict(
    electricity = df.BPS_Electric_Bill/df.BPS_Electric_Bill.max()*20,
    colors = df.SMMA_FA_Existing_PV.apply(lambda x: 'red' if x=="No" else 'green'),
    label = df.SMMA_FA_Existing_PV.apply(lambda x: 'Without PV' if x=="No" else 'With PV'),
    water = df["BPS_Water_Bill"],
    gas = df["BPS_Gas_Bill"]
    ))

# We either add the array of sizes to the data_source object or should avoid its usage for now
p = figure(plot_width=400, plot_height=400, title="Expenses of the buildings")
p.circle(x='water', y='gas', size='electricity', color='colors', alpha=0.5,
        legend_group='label', source=data_source)
p.xaxis.formatter = NumeralTickFormatter(format='$0,0'); p.xaxis.axis_label = "Water Bill"
p.yaxis.formatter = NumeralTickFormatter(format='$0,0'); p.yaxis.axis_label = "Gas Bill"
show(p)

### How could we add a 5th dimension? 
* directions?
* common region?

### Add html tags

In [38]:
desc = Div(text="Description of the plot: Not many buildings have Photovoltaic panels...")
largetitle = Div(text="<h2> Infographics of School Buildings</h2>")

In [39]:
show(Column(largetitle, Row(p,desc)))